In [ ]:
def fetch_wikidata_refined(identifiers, exchange_qid, mode="isin"):
    url = 'https://query.wikidata.org/sparql'
    
    if mode == "isin":
        values_formatted = ' '.join([f'"{i}"' for i in identifiers])
        lookup_logic = "?company wdt:P946 ?reqId . BIND(?reqId AS ?isin)"
    else:
        values_formatted = ' '.join([f'"{t}"' for t in identifiers])
        lookup_logic = f"?company p:P414 [ ps:P414 wd:{exchange_qid} ; pq:P249 ?reqId ] . OPTIONAL {{ ?company wdt:P946 ?isin . }}"

    query = f"""
    SELECT ?reqId ?company ?companyLabel ?isin ?foundingYear
      (GROUP_CONCAT(DISTINCT ?sub_info; separator="|") AS ?subs_combined)
      (GROUP_CONCAT(DISTINCT ?ind_info; separator="|") AS ?inds_combined)
      (GROUP_CONCAT(DISTINCT ?inv_info; separator="|") AS ?invs_combined)
      (GROUP_CONCAT(DISTINCT ?prod_info; separator="|") AS ?prods_combined)
      (GROUP_CONCAT(DISTINCT ?area_info; separator="|") AS ?areas_combined)
      (GROUP_CONCAT(DISTINCT ?loc_info; separator="|") AS ?locs_combined)
      (GROUP_CONCAT(DISTINCT ?inst_info; separator="|") AS ?insts_combined)
      (GROUP_CONCAT(DISTINCT ?part_info; separator="|") AS ?parts_combined)
      (GROUP_CONCAT(DISTINCT ?owner_info; separator="|") AS ?owners_combined)
    WHERE {{
      VALUES ?reqId {{ {values_formatted} }}
      {lookup_logic}
      
      OPTIONAL {{ ?company wdt:P571 ?fDate . BIND(YEAR(?fDate) AS ?foundingYear) }}
      
      # Struktur: QID und Label werden vor dem GROUP_CONCAT kombiniert, das ist effizienter
      OPTIONAL {{ ?company wdt:P355 ?sub . ?sub rdfs:label ?subL . FILTER(LANG(?subL) = "en") BIND(CONCAT(REPLACE(STR(?sub), ".*Q", "Q"), ":", ?subL) AS ?sub_info) }}
      OPTIONAL {{ ?company wdt:P452 ?ind . ?ind rdfs:label ?indL . FILTER(LANG(?indL) = "en") BIND(CONCAT(REPLACE(STR(?ind), ".*Q", "Q"), ":", ?indL) AS ?ind_info) }}
      OPTIONAL {{ ?company wdt:P1830 ?inv . ?inv rdfs:label ?invL . FILTER(LANG(?invL) = "en") BIND(CONCAT(REPLACE(STR(?inv), ".*Q", "Q"), ":", ?invL) AS ?inv_info) }}
      OPTIONAL {{ ?company wdt:P1056 ?prod . ?prod rdfs:label ?prodL . FILTER(LANG(?prodL) = "en") BIND(CONCAT(REPLACE(STR(?prod), ".*Q", "Q"), ":", ?prodL) AS ?prod_info) }}
      OPTIONAL {{ ?company wdt:P2541 ?area . ?area rdfs:label ?areaL . FILTER(LANG(?areaL) = "en") BIND(CONCAT(REPLACE(STR(?area), ".*Q", "Q"), ":", ?areaL) AS ?area_info) }}
      OPTIONAL {{ ?company wdt:P276 ?loc . ?loc rdfs:label ?locL . FILTER(LANG(?locL) = "en") BIND(CONCAT(REPLACE(STR(?loc), ".*Q", "Q"), ":", ?locL) AS ?loc_info) }}
      OPTIONAL {{ ?company wdt:P31 ?inst . ?inst rdfs:label ?instL . FILTER(LANG(?instL) = "en") BIND(CONCAT(REPLACE(STR(?inst), ".*Q", "Q"), ":", ?instL) AS ?inst_info) }}
      OPTIONAL {{ ?company wdt:P361 ?part . ?part rdfs:label ?partL . FILTER(LANG(?partL) = "en") BIND(CONCAT(REPLACE(STR(?part), ".*Q", "Q"), ":", ?partL) AS ?part_info) }}
      OPTIONAL {{ ?company wdt:P127 ?owner . ?owner rdfs:label ?ownerL . FILTER(LANG(?ownerL) = "en") BIND(CONCAT(REPLACE(STR(?owner), ".*Q", "Q"), ":", ?ownerL) AS ?owner_info) }}

      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    GROUP BY ?reqId ?company ?companyLabel ?isin ?foundingYear
    """
    
    headers = {'User-Agent': 'PortfolioBot/1.0 (contact: deine-mail@beispiel.de)', 'Accept': 'application/sparql-results+json'}
    try:
        response = requests.post(url, data={'query': query, 'format': 'json'}, headers=headers)
        response.raise_for_status()
        data = response.json()
        
        parsed_results = []
        for item in data['results']['bindings']:
            # Hilfsfunktion zum Trennen von QID und Label
            def split_combined(val):
                if not val: return "", ""
                parts = val.split('|')
                qids = [p.split(':', 1)[0] for p in parts if ':' in p]
                labels = [p.split(':', 1)[1] for p in parts if ':' in p]
                return "|".join(labels), "|".join(qids)

            res = {
                'requested_id': item.get('reqId', {}).get('value'),
                'company_qid': item.get('company', {}).get('value').split('/')[-1],
                'name': item.get('companyLabel', {}).get('value'),
                'isin': item.get('isin', {}).get('value'),
                'founding_year': item.get('foundingYear', {}).get('value')
            }
            
            # Mapping der kombinierten Felder zurück in separate Spalten
            res['subsidiaries'], res['subsidiary_qids'] = split_combined(item.get('subs_combined', {}).get('value'))
            res['industries'], res['industry_qids'] = split_combined(item.get('inds_combined', {}).get('value'))
            res['investments'], res['investment_qids'] = split_combined(item.get('invs_combined', {}).get('value'))
            res['products'], res['product_qids'] = split_combined(item.get('prods_combined', {}).get('value'))
            res['operating_areas'], res['operating_area_qids'] = split_combined(item.get('areas_combined', {}).get('value'))
            res['locations'], res['location_qids'] = split_combined(item.get('locs_combined', {}).get('value'))
            res['instances'], res['instance_qids'] = split_combined(item.get('insts_combined', {}).get('value'))
            res['parts_of'], res['part_of_qids'] = split_combined(item.get('parts_combined', {}).get('value'))
            res['owners'], res['owner_qids'] = split_combined(item.get('owners_combined', {}).get('value'))
            
            parsed_results.append(res)
        return parsed_results
    except Exception as e:
        print(f"Fehler im Modus {mode}: {e}")
        return []

In [ ]:
lookup_logic = "?company wdt:P946 ?reqId . BIND(?reqId AS ?isin)"
lookup_logic = f"?company p:P414 [ ps:P414 wd:{batch[1]} ; pq:P249 ?reqId ] . OPTIONAL {{ ?company wdt:P946 ?isin . }}"
def FetchWikiData(self, batch, url='https://query.wikidata.org/sparql'):
        if batch[0] == "ISIN":
            values_formatted = ' '.join([f'"{i}"' for i in batch[2]['ISIN']])
            lookup_logic = """OPTIONAL { ?company wdt:P946 ?reqId . } BIND(?reqId AS ?isin)"""
        else:
            values_formatted = ' '.join([f'"{t}"' for t in batch[2]['OriginalTicker']])
            lookup_logic = f"""OPTIONAL {{ ?company p:P414 [ ps:P414 wd:{batch[1]} ; pq:P249 ?reqId ] . }} OPTIONAL {{ ?company wdt:P946 ?isin . }}"""
        query = f"""
        SELECT ?reqId ?company ?companyLabel ?isin ?foundingYear
        (GROUP_CONCAT(DISTINCT ?subLabel; separator="|") AS ?subsidiaries)
        (GROUP_CONCAT(DISTINCT ?subID; separator="|") AS ?subsidiary_IDs)
        (GROUP_CONCAT(DISTINCT ?indLabel; separator="|") AS ?industries)
        (GROUP_CONCAT(DISTINCT ?indID; separator="|") AS ?industry_IDs)
        (GROUP_CONCAT(DISTINCT ?compLabel; separator="|") AS ?competitors)
        (GROUP_CONCAT(DISTINCT ?compID; separator="|") AS ?competitor_IDs)
        (GROUP_CONCAT(DISTINCT ?invLabel; separator="|") AS ?investments)
        (GROUP_CONCAT(DISTINCT ?invID; separator="|") AS ?investment_IDs)
        (GROUP_CONCAT(DISTINCT ?prodLabel; separator="|") AS ?products)
        (GROUP_CONCAT(DISTINCT ?prodID; separator="|") AS ?product_IDs)
        (GROUP_CONCAT(DISTINCT ?areaLabel; separator="|") AS ?operating_areas)
        (GROUP_CONCAT(DISTINCT ?areaID; separator="|") AS ?operating_area_IDs)
        (GROUP_CONCAT(DISTINCT ?locLabel; separator="|") AS ?locations)
        (GROUP_CONCAT(DISTINCT ?locID; separator="|") AS ?location_IDs)
        (GROUP_CONCAT(DISTINCT ?instLabel; separator="|") AS ?instances)
        (GROUP_CONCAT(DISTINCT ?instID; separator="|") AS ?instance_IDs)
        (GROUP_CONCAT(DISTINCT ?partLabel; separator="|") AS ?parts_of)
        (GROUP_CONCAT(DISTINCT ?partID; separator="|") AS ?part_of_IDs)
        (GROUP_CONCAT(DISTINCT ?ownerLabel; separator="|") AS ?owners)
        (GROUP_CONCAT(DISTINCT ?ownerID; separator="|") AS ?owned_by_IDs)
        
        WHERE {{
        VALUES ?reqId {{ {values_formatted} }}
        {lookup_logic}
        
        OPTIONAL {{ ?company wdt:P571 ?fDate . BIND(YEAR(?fDate) AS ?foundingYear) }}
        
        # Tochtergesellschaften (P355) & Unternehmensbeteiligungen (P1830)
        OPTIONAL {{ 
            ?company (wdt:P355) ?sub . ?sub rdfs:label ?subLabel . FILTER(LANG(?subLabel) = "en") 
            BIND(REPLACE(STR(?sub), ".*Q", "Q") AS ?subID)
        }}
        # Beteiligungen (Investments)
        OPTIONAL {{ 
            ?company (wdt:P1830) ?inv . ?inv rdfs:label ?invLabel . FILTER(LANG(?invLabel) = "en") 
            BIND(REPLACE(STR(?inv), ".*Q", "Q") AS ?invID)
        }}
        # Industrien (P452)
        OPTIONAL {{ 
            ?company wdt:P452 ?ind . ?ind rdfs:label ?indLabel . FILTER(LANG(?indLabel) = "en") 
            BIND(REPLACE(STR(?ind), ".*Q", "Q") AS ?indID)
        }}
        # Produkte / Material Produced (P1056)
        OPTIONAL {{ 
            ?company wdt:P1056 ?prod . ?prod rdfs:label ?prodLabel . FILTER(LANG(?prodLabel) = "en") 
            BIND(REPLACE(STR(?prod), ".*Q", "Q") AS ?prodID)
        }}
        # Operating Area (P2541)
        OPTIONAL {{ 
            ?company wdt:P2541 ?area . ?area rdfs:label ?areaLabel . FILTER(LANG(?areaLabel) = "en") 
            BIND(REPLACE(STR(?area), ".*Q", "Q") AS ?areaID)
        }}
        # Location (P276)
        OPTIONAL {{ 
            ?company wdt:P276 ?loc . ?loc rdfs:label ?locLabel . FILTER(LANG(?locLabel) = "en") 
            BIND(REPLACE(STR(?loc), ".*Q", "Q") AS ?locID)
        }}
        # Instance of (P31)
        OPTIONAL {{ 
            ?company wdt:P31 ?inst . ?inst rdfs:label ?instLabel . FILTER(LANG(?instLabel) = "en") 
            BIND(REPLACE(STR(?inst), ".*Q", "Q") AS ?instID) 
        }}
        # Part of (P361)
        OPTIONAL {{ 
            ?company wdt:P361 ?part . ?part rdfs:label ?partLabel . FILTER(LANG(?partLabel) = "en") 
            BIND(REPLACE(STR(?part), ".*Q", "Q") AS ?partID)
        }}
        # Owned by (P127)
        OPTIONAL {{ 
            ?company wdt:P127 ?owner . ?owner rdfs:label ?ownerLabel . FILTER(LANG(?ownerLabel) = "en") 
            BIND(REPLACE(STR(?owner), ".*Q", "Q") AS ?ownerID)
        }}

        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en,fr,de,it". }}
        }}
        GROUP BY ?reqId ?company ?companyLabel ?isin ?foundingYear
        """
        headers = {'User-Agent': 'PortfolioBot/1.0', 'Accept': 'application/sparql-results+json'}
        print(query)
        try:
            response = requests.post(url, data={'query': query, 'format': 'json'}, headers=headers)
            response.raise_for_status()
            data = response.json()
            return data
        except Exception as e:
            print(f"Fehler für Kriterium {batch[0]} , QID {batch[1]}: {e}")
            return []
        

In [ ]:
'''
    def WikiDataQID(self, batches):
        all_batches_results = [] # Die Liste, die am Ende Batches enthält
        
        for batch in batches:
            current_batch_list = [] # Hier sammeln wir die Dicts NUR für diesen einen Batch
            
            if batch[0] == "ISIN":
                values_formatted = ' '.join([f'"{i}"' for i in batch[2]['ISIN']])
                query = f"""SELECT ?reqId ?company WHERE {{ VALUES ?reqId {{ {values_formatted} }} OPTIONAL {{ ?company wdt:P946 ?reqId . }} }}"""
                data = self.InvokeWikiData(query)         
                results = data['results']['bindings']
                
                for result in results:
                    reqId = result['reqId']['value']
                    company = result.get('company', {}).get('value', None)
                    company_qid = company.split('/')[-1] if company else None
                    # Hinzufügen zum aktuellen Batch
                    current_batch_list.append({"RequestID": reqId, "Company_QID": company_qid})
            
            elif batch[0] == 'Ticker':
                tickers_formatted = ' '.join([f'"{i}"' for i in batch[2]['OriginalTicker']])
                query = f"""SELECT ?reqId ?company WHERE {{ VALUES ?reqId {{ {tickers_formatted} }} OPTIONAL {{ ?company p:P414 [ ps:P414 wd:{batch[1]} ; pq:P249 ?reqId ] . }} }}"""
                data = self.InvokeWikiData(query)         
                results = data['results']['bindings']
                
                for result in results:
                    reqId = result['reqId']['value']
                    company = result.get('company', {}).get('value', None)
                    company_qid = company.split('/')[-1] if company else None
                    # Hinzufügen zum aktuellen Batch
                    current_batch_list.append({"RequestID": reqId, "Company_QID": company_qid})
            
            # WICHTIG: Den fertigen Batch der Hauptliste hinzufügen
            if current_batch_list:
                all_batches_results.append(current_batch_list)
                
        return all_batches_results
        '''